# Module 1 - Langchain Core
## 1.2  Input,Output,Prompt Engineering


> Goals
  1.   To format inputs and outputs-how agents and chains talk
  2.   How to format conversation for clarity

Consider we are building a multi-agent or multi-chain system. And output of one agent is input of another. Now we want output of first agent in a specific format. So we have to use our prompt in specific format using ```ResponseSchema``` and ```StructuredOutputParser```.

So we will build a 3 mini agent to apply these things-
1. Tracker Agent :

    Input: user says their LMP (last menstrual period)

    Output: {weeks_pregnant: int, trimester: str}

2. Weekly Info Agent :

    Input: number of weeks

    Output: {baby_development: str, maternal_tip: str}

    Use partial() to lock role as "pregnancy educator"

3. Symptom Agent :

    Input: a symptom

    Output: {advice: str, warning: str}

    Use structured output parsing.

So we will use ```partial()```,```ResponseSchema``` and ```StructuredOutputParser``` to build such a chain.

In [1]:
pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.4 MB/s eta 0:00:00


In [2]:
# Setting up environment
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Your API Key:")

Your API Key:··········


In [9]:
# necessary imports
from langchain_core.runnables import RunnableLambda,RunnableBranch
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
from datetime import datetime

# model initialisation
det_model = init_chat_model("llama3-8b-8192", model_provider="groq",temperature = 0)
model = init_chat_model("llama3-8b-8192", model_provider="groq")

# calculating gestation weeks
def gestation_weeks(input : dict , today_str=None):

  if today_str is None:
    today = datetime.now()
  else:
    today = datetime.strptime(today_str,"%d%m%Y")

  try:
    lmp_date = input["date"]
    lmp_date = datetime.strptime(lmp_date,"%d%m%Y")
    if lmp_date > today:
        return {"error": "LMP date is in the future."}
    diff = today - lmp_date
    weeks = diff.days // 7
    return {"weeks": weeks}
  except ValueError:
    return {"error": "Invalid date format."}

gst_weeks = RunnableLambda(gestation_weeks)

# creating tracker_chain
schemas_1 = [
    ResponseSchema(name='weeks', description="number of weeks"),
    ResponseSchema(name='trimester', description="Trimester based on number of weeks")
]

parser_1 = StructuredOutputParser.from_response_schemas(schemas_1)
format_1 = parser_1.get_format_instructions()

trimester_prompt = ChatPromptTemplate.from_template("""
You are a medical assistant.
Decide the pregnancy trimester based on weeks.

Logics:
- First trimester: 0–13 weeks
- Second trimester: 14–27 weeks
- Third trimester: 28+ weeks

{format_instructions}

Example:
Weeks: 21
Output: {{"trimester": "second"}}

Now, classify:
Weeks: {weeks}
""").partial(format_instructions=format_1)

tracker_chain = gst_weeks | trimester_prompt | det_model | parser_1
# tracker_chain is working perfectly

# Weekly info agent
schemas_2 = [
    ResponseSchema(name='baby_development_stage', description="baby developement based on weeks"),
    ResponseSchema(name='maternity_tip', description="tips for mother")
]

parser_2 = StructuredOutputParser.from_response_schemas(schemas_2)
format_2 = parser_2.get_format_instructions()

information_prompt = ChatPromptTemplate.from_template("""
you are obstetrician-gynecologist.
Give necessary information about developement of baby and tips and advice for mother based on number of weeks of pregnancy.

{format_instructions}
Now Weeks: {weeks}
""").partial(format_instructions=format_2)

information_chain = information_prompt | model | parser_2
# creating link between them
chain = tracker_chain | information_chain

# weekly info agent is working properly

# symptom agent
schemas_3 = [
    ResponseSchema(name='advice', description="advice based on symptoms"),
    ResponseSchema(name='warning', description="warning based on symptoms")
]

parser_3 = StructuredOutputParser.from_response_schemas(schemas_3)
format_3 = parser_3.get_format_instructions()

symptom_prompt = ChatPromptTemplate.from_template("""
You are a gynecologist.
Give necessary,reliable and safe advice and warning based on symptoms.

{format_instructions}
Now symptoms: {symptoms}
""").partial(format_instructions=format_3)

symptom_chain = symptom_prompt | model | parser_3


In [17]:
result = chain.invoke({"date": "23022025"})
print("Information about baby in this stage:")
print(result["baby_development_stage"])
print("Tips for mother:")
print(result["maternity_tip"])

# checking for symptom
user_input = input("Do you have any symptom? (yes/no): ")
if user_input.lower() == "yes":
  symptom = input("Give your symptoms: ")
  symptom_result = symptom_chain.invoke({"symptoms" : symptom})

  print("Advice:")
  print(symptom_result["advice"])
  print("Warning:")
  print(symptom_result["warning"])

else:
  print("Thank you,very much. Have a safe day.")

Information about baby in this stage:
The embryo's major organs and body systems are forming. The heart is pumping blood through its chambers, and the lungs are starting to produce surfactant, a substance that helps them expand and contract properly after birth. The baby's nervous system is also developing, and its brain is producing essential neurotransmitters.
Tips for mother:
At 16 weeks, it's essential to maintain a healthy diet rich in essential nutrients. Include foods high in folic acid, iron, and calcium in your meals. Also, try to stay hydrated by drinking plenty of water throughout the day. As your belly grows, make sure to wear comfortable clothing and take regular breaks to rest your back.
Do you have any symptom? (yes/no): yes
Give your symptoms: leg ache
Advice:
If you're experiencing leg ache, it's possible that it may be related to your menstrual cycle or hormonal changes. I recommend keeping a symptom journal to track when the ache occurs and its severity. In the meant